<a href="https://colab.research.google.com/github/NbtKmy/gc_workshops/blob/main/Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI-Whisper
Whisper ist ein Spracherkennungsmodel, das von OpenAI entwickelt worden ist. 
Mit Whisper kann man zum Beispiel eine gesprochene Sprache (Audio-Datei) in einen Text umwandeln (transkribieren).

Im [Github-Repository](https://github.com/openai/whisper) findet man die Code und Beschreibungen. 

## Vor der Installation
Vor der Installation überprüfen wir zuerst, ob GPU (Graphicprozessor) für dieses Notebook freigeschaltet ist. Viele KI-Modelle, darunter auch Whisper, braucht GPU.

In [ ]:
!nvidia-smi

Wenn der Befehl oben fehlgeschlagen ist, dann muss man die Einstellung ändern. 
"Bearbeiten"-"Notebook Einstellungen" klicken. Dann im geöffneten Popup bei "Hardwarebeschleuniger" "GPU" anwählen und Speichern.

Google bietet schnelle Prozessor wie GPU an, aber nur bis zur gewissen Rechnungskapazität und bis zum gewissen Laufzeit des Prozessors. Wenn man noch mehr Rechnungskapazität will, kann man entweder die Angebote von unterschiedlichen Anbietern beziehen oder Graphic Board selber anschaffen und in dem eigenen PC einbauen.

Die [**Science IT**](https://www.zi.uzh.ch/en/teaching-and-research/science-it/computing.html) an der Universität Zürich bietet ebenso die Dienstleistung für solche GPU an.

### Version Check von Tensorflow (für Fehlermeldung)

In diesem Abschnitt überprüfen wir die Version von Tensorflow. 
Zuerst den folgenden Befehl laufen lassen:

In [ ]:
!python -c "import tensorflow as tf; print(tf.__version__)"

Falls jetzt die Fehlermeldung auftaucht, sieht es aus, dass GPU nicht richtig zugegriffen wird:

```
2023-02-24 09:25:23.842385: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 09:25:25.054473: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-24 09:25:25.054576: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-24 09:25:25.054596: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
```

>Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly

Das Problem scheint, dass Tensorflow irgendwie Version 7 von libnvinfer.so.7 und libnvinfer_plugin.so.7 suchen geht (und nicht findet). 

Wenn Tensorflow jetzt GPU zugreifen kann, muss man den folgenden Befehl nicht ausführen.




In [ ]:
# wenn die Version von Tersorflow 2.11.0 ist und Fehlermeldung erzeugt wird, dann Tensorflow downgraden
# In diesem Prozess ergeben sich einige Errors wegen der Abhängigkeiten der Libraries & Packages.
# Aber nach Ausführung des folgenden Befehls taucht das GPU-Probleme nicht mehr auf...

!pip install tensorflow-gpu==2.5.0

In [ ]:
!python -c "import tensorflow as tf; print(tf.__version__)"

## Installieren


In [ ]:
!pip install git+https://github.com/openai/whisper.git

Dann ffmpeg installieren 

In [ ]:
!sudo apt update && sudo apt install ffmpeg

## Einfache Anwendung durch Commandline

Zuerst ein Audio-File (.flac, .mp3 oder .wav) hochladen!



Bei der Anwendung kann man unterschiedliche Modelle auswählen. 

Laut der [offziellen Angabe](https://github.com/openai/whisper#available-models-and-languages) gibt es 5 multi-linguale Modelle.

Je nach der Sprache ergeben sich unterschiedliche Fehlerquoten. 

In [ ]:
from google.colab import files

uploaded = files.upload()
uploaded_filename = list(uploaded.keys())[0]
# Unter dem Variabel uploaded_filename ist der Pfad zu dem hochgeladenen File untergebracht

In [ ]:
# Audio anhören
from IPython.display import Audio
Audio(uploaded_filename)

In [ ]:
!whisper $uploaded_filename --model medium

In [ ]:
!whisper $uploaded_filename --task translate

## In einer Python-Code verwenden

In [ ]:
import whisper

model = whisper.load_model('medium')
model.device
result = model.transcribe(uploaded_filename)
print(result['text'])

Mit der Funktion "transcribe()" wird die Audio-Eingabe pro 30 sec. getrennt und Stück für Stück verarbeitet:

>Internally, the transcribe() method reads the entire file and processes the audio with a sliding 30-second window, performing autoregressive sequence-to-sequence predictions on each window.
(aus: [Python usage](https://github.com/openai/whisper#python-usage))

Mit der Funktion "transcribe()" kann man deshalb eine Audio-Eingabe länger als 30 sec. am Stück verarbeiten. 
```
model = whisper.load_model([model_size])
audio = whisper.load_audio([filename])
result = model.transcribe(audio, verbose=True, temperature=0.8, language=[lang])
```

Wenn man die andere Funktion "decode()" verwendet, muss die Länge der Audio-Eingabe max. 30 sec. sein. Wenn die Audio-Aufnahme länger als 30 sec. ist, braucht man ein wenig creativ zu sein. Siehe unten... 

In [ ]:
import whisper

fileName = 'result_whisper'#@param {type:'string'}
lang = 'de'#@param ['ja', 'en', 'de']
model_size = 'medium'#@param ['tiny', 'base', 'small', 'medium', 'large']
model = whisper.load_model(model_size)


audio = whisper.load_audio(uploaded_filename)
outputTextsArr = []

while audio.size > 0:
    tirmedAudio = whisper.pad_or_trim(audio)
    startIdx = tirmedAudio.size
    audio = audio[startIdx:]

    mel = whisper.log_mel_spectrogram(tirmedAudio).to(model.device)

    options = whisper.DecodingOptions(language=lang, without_timestamps=True)
    result = whisper.decode(model, mel, options)
    outputTextsArr.append(result.text)

outputTexts = ' '.join(outputTextsArr)
print(outputTexts)


with open(f'{fileName}.txt', 'w') as f:
    f.write(outputTexts)

## Ein YouTube-Video mit Untertitel oder Übersetzung versehen

Um ein YouTube-Video herunterzuladen, verwendet man hier [yt-dlp](https://pypi.org/project/yt-dlp/).


In [ ]:
!pip install -U yt-dlp

In [ ]:
from yt_dlp import YoutubeDL

yt_url = 'https://youtu.be/Ifkhj0XtGGM'#@param {type:'string'}
vd_filename = 'test'#@param{type:'string'} 
vd_filename += '.mp4'

ydl_opts = {'format': 'best',
            'outtmpl': vd_filename }
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([yt_url])

...Überprüfen, ob das Video tatsächlich heruntergeladen ist...

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open(vd_filename, 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [ ]:
!pip install -U srt
# ffmpeg-python ist bereits mit whisper installiert...
# !pip install ffmpeg-python

In [ ]:
import whisper
from datetime import timedelta
from srt import Subtitle
import srt



# Text umbrechen, wenn eine Zeile lang ist　... wahrscheinlich braucht man nicht...
def add_line(line):
    new_line = line
    space_count = new_line.count(' ')
    list_line = list(new_line)
    space_max = 4
    if space_count > space_max :
        space_index = [i for i, x in enumerate(list_line) if x == ' ']
        new_line = line[:space_index[space_max]] + '\n' + line[space_index[space_max]:]

    return new_line

# Übersetzung erstellen
model = whisper.load_model('medium')
result = model.transcribe(vd_filename, task='translate')

segments = result['segments']
 
subtitles = []
 
for segment in segments:
    index = segment['id'] + 1
    start = segment['start']
    end = segment['end']
    text = add_line(segment['text'])
    subtitle = Subtitle(
                   index=index, 
                   start=timedelta(seconds=timedelta(seconds=start).seconds, microseconds=timedelta(seconds=start).microseconds),
                   end=timedelta(seconds=timedelta(seconds=end).seconds, microseconds=timedelta(seconds=end).microseconds), 
                   content=text, 
                   proprietary=''
              )
    subtitles.append(subtitle)

# Untertitel speichern 
with open('subtitle.srt', mode='w', encoding='utf-8') as f:
    f.write(srt.compose(subtitles))



In [ ]:
import ffmpeg

# Das Video mit Untertitel versehen und speichern
video = ffmpeg.input(vd_filename)
audio = video.audio
ffmpeg.concat(video.filter('subtitles', 'subtitle.srt'), audio, v=1, a=1).output('output_vid.mp4').run()

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('output_vid.mp4', 'rb').read()
vid_wSub = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{vid_wSub}" type="video/mp4">
</video>""")